This document provides an in-depth explanation of the code that transpiles a quantum circuit into a new circuit using a constrained set of basis gates: {RZ, SX, X, CX, ID}. The transpilation process involves decomposing both single-qubit and two-qubit gates into these basis gates, ensuring that the resulting circuit is compatible with specific quantum hardware requirements.



#Code Overview
The code consists of three key parts:

Decomposition of Single-Qubit Gates into Basis Gates
Decomposition of Two-Qubit Gates into Basis Gates
Transpilation of a Full Circuit into the Target Gate Basis

In [2]:
!pip install qiskit
!pip install qiskit_aer
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=40db5737f25138f8f0a9ed36fdc923390554e4988c288f6dce3210d7b485922d
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [3]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator, Statevector
from qiskit.visualization import plot_histogram

%matplotlib inline

In [ ]:
import numpy as np
from qiskit import QuantumCircuit

# Target gate basis
TARGET_BASIS = ['rz', 'sx', 'x', 'cx', 'id']

#Single-Qubit Gate Conversion

Explanation:
This function handles the decomposition of single-qubit gates into the target gate basis.



*   The Hadamard gate is decomposed into a sequence of basis gates as:H=Rz(π)×SX×Rz(π) \\
*   The RX gate is parameterized by an angle θ and is decomposed as:
RX(θ)=Rz(−π/2)×SX×Rz(θ+π)×SX×Rz(π/2)


*   The RY gate is decomposed using the X, RZ, and SX gates:

  RY(θ)=X×Rz(π/2​)×SX×Rz(θ)×SX×Rz(−π/2)
*   Both RZ and X gates are part of the target basis, so they are used directly.




*   The phase gate P(θ) is implemented as an RZ(θ) since both change the phase of a qubit.


Decomposition Goal:
The goal is to reduce complex gates into a series of simpler gates that can be directly implemented on quantum hardware that supports only {RZ, SX, X, CX, ID}.

In [ ]:
def convert_single_qubit_gate(gate_name, qubit, params):
    """
    Converts single-qubit gates to {RZ, SX, X, ID} basis.

    Parameters:
        gate_name (str): The name of the gate.
        qubit (int): The index of the qubit the gate acts on.
        params (list): The parameters of the gate, if any.

    Returns:
        QuantumCircuit: A circuit containing the equivalent gate in the target basis.
    """
    new_circuit = QuantumCircuit(1)
    qubit = qubit -1

    if gate_name == 'h':
        # Decompose Hadamard: H = Rz(pi) * SX * Rz(pi)
        new_circuit.rz(np.pi, 0)
        new_circuit.sx(0)
        new_circuit.rz(np.pi, 0)
        new_circuit.barrier()


    elif gate_name == 'rx':
        # Decompose RX(theta): RX = Rz(-pi/2) * SX * Rz(theta + pi) * SX * Rz(pi/2)
        theta = params[0]
        new_circuit.rz(-np.pi/2, 0)
        new_circuit.sx(0)
        new_circuit.rz(theta + np.pi, 0)

        new_circuit.sx(0)
        new_circuit.rz(np.pi/2, 0)
        # new_circuit.barrier(1)


    elif gate_name == 'ry':
        # Decompose RY(theta) = X*RZ(-pi/2)*SX*RZ(theta)*SX*RZ(-pi/2)
        theta = params[0]
        new_circuit.x(0)
        new_circuit.rz(np.pi/2, 0)
        new_circuit.sx(0)
        new_circuit.rz(theta, 0)
        new_circuit.sx(0)
        new_circuit.rz(-np.pi/2, 0)
        # new_circuit.barrier(1)



    elif gate_name == 'rz':
        # RZ gate is part of the target basis, return as is
        new_circuit.rz(params[0], 0)
        # new_circuit.barrier(1)


    elif gate_name == 'x':
        # X gate is part of the target basis, return as is
        new_circuit.x(0)
        # new_circuit.barrier(1)

    elif gate_name == 'p':
        theta = params[0]
        new_circuit.rz(theta,0)
    else:
        raise ValueError(f"Single-qubit gate '{gate_name}' is not supported.")

    return new_circuit

#convert_two_qubit_gate
Explanation:
This function decomposes two-qubit gates into the CX gate (from the target basis) and other single-qubit gates.

CX Gate:

Since CX is already part of the target basis, it is directly used in the new circuit.
CZ Gate:

The CZ gate is decomposed using the CX and Hadamard-like gates:
CZ=H×CX×H
In this case, Hadamard (H) is implemented using RZ and SX gates as explained earlier.
SWAP Gate:

The SWAP gate is decomposed using three consecutive CX gates.
Controlled Rotation Gates (CRX, CRY, CRZ):

These gates can also be decomposed into a combination of RZ, CX, and rotation gates, though the exact decompositions are not implemented in this snippet.
Decomposition Goal:
The aim is to reduce complex two-qubit gates into CX gates (the only two-qubit gate in the target basis) and other single-qubit gates.

In [100]:
# Function to convert two-qubit gates to target basis (only CX is allowed)
def convert_two_qubit_gate(gate_name, control_qubit, target_qubit,n,param):
    """
    Converts two-qubit gates to {CX} basis.

    Parameters:
        gate_name (str): The name of the gate.
        control_qubit (int): The index of the control qubit.
        target_qubit (int): The index of the target qubit.

    Returns:
        QuantumCircuit: A circuit containing the equivalent gate in the target basis.
    """

    new_circuit = QuantumCircuit(n)

    if gate_name == 'cx':
        # CX gate is part of the target basis, return as is
        new_circuit.cx(control_qubit, target_qubit)
        # new_circuit.barrier(n-1)


    elif gate_name == 'cz':
        # Decompose CZ gate using H and CX: CZ = H * CX * H
        new_circuit.rz(np.pi, target_qubit)
        new_circuit.sx(target_qubit)
        new_circuit.rz(np.pi, target_qubit)
        new_circuit.cx(control_qubit, target_qubit)
        new_circuit.rz(np.pi, target_qubit)
        new_circuit.sx(target_qubit)
        new_circuit.rz(np.pi, target_qubit)
    elif gate_name == 'swap':
        # Swap gate is part of the target basis, return as is
        new_circuit.cx(control_qubit, target_qubit)
        new_circuit.cx(target_qubit, control_qubit)
        new_circuit.cx(control_qubit, target_qubit)

    elif gate_name == 'crx':
        theta = param[0]
        # new_circuit.rx(theta/2, target_qubit)
        # new_circuit.cx(control_qubit, target_qubit)


    elif gate_name == 'cry':
        theta = param[0]
        new_circuit.ry(theta/2, target_qubit)
        new_circuit.cx(control_qubit, target_qubit)
        new_circuit.ry(-theta/2, target_qubit)
        new_circuit.cx(control_qubit, target_qubit)

    elif gate_name == 'crz':
        theta = param[0]
        new_circuit.rz(theta/2, target_qubit)
        new_circuit.cx(control_qubit, target_qubit)
        new_circuit.rz(-theta/2, target_qubit)
        new_circuit.cx(control_qubit, target_qubit)

    elif gate_name == 'cp':
      if(control_qubit > target_qubit):
        theta = param[0]
        new_circuit.rz(theta,n-1)
        new_circuit.cx(n-1,0)
        new_circuit.rz(theta,0)
        new_circuit.cx(n-1,0)
        new_circuit.rz(theta,0)
        # new_circuit.barrier(n-1)
      elif(control_qubit < target_qubit):
        theta = param[0]
        new_circuit.rz(theta,0)
        new_circuit.cx(0,n-1)
        new_circuit.rz(theta,n-1)
        new_circuit.cx(0,n-1)
        new_circuit.rz(theta,n-1)
        # new_circuit.barrier(n-1)

    else:
        raise ValueError(f"Two-qubit gate '{gate_name}' is not supported.")

    return new_circuit

#transpile_circuit
This is the main function responsible for transpiling the original quantum circuit into a new circuit that only uses the target gate basis.

Iterate Through Instructions:

The function iterates through each instruction (gate) in the original circuit. For each gate: \\
Single-Qubit Gates: It calls the convert_single_qubit_gate() function to decompose the gate and compose it into the new circuit. \\
Two-Qubit Gates: It calls the convert_two_qubit_gate() function to decompose the gate into CX and other gates. \\
Qubit Index Mapping:

The function ensures that the qubits are mapped correctly in the new circuit, accounting for both single-qubit and multi-qubit gates. \\
Result:

The function returns a new QuantumCircuit object that uses only the target basis gates. \\
Transpilation Goal: \\
The final goal is to transform any given circuit into a format that can be executed on quantum hardware that supports only a limited set of basis gates.



In [83]:
def transpile_circuit(circuit):
    """
    Transpiles the given quantum circuit to use only the target gate basis.

    Parameters:
        circuit (QuantumCircuit): The quantum circuit to be transpiled.

    Returns:
        QuantumCircuit: A new circuit transpiled into the target gate basis.
    """
    # Create a new QuantumCircuit with the same number of qubits and classical bits
    new_circuit = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)

    # Iterate over each instruction in the original circuit
    for instr, qubits, clbits in circuit.data:
        gate_name = instr.name
    # Use find_bit() to get the qubit index for each qubit
        qubit_indices = [qc.find_bit(qubit).index for qubit in qubits] # Extract qubit indices
        params = instr.params

        # Check if it's a single-qubit gate or two-qubit gate
        if len(qubit_indices) == 1:
            # Single-qubit gate
            single_qubit_circuit = convert_single_qubit_gate(gate_name, qubit_indices[0], params)
            new_circuit = new_circuit.compose(single_qubit_circuit, qubits=qubit_indices)

        elif len(qubit_indices) == 2:
            # Two-qubit gate
            a = min(qubit_indices[0],qubit_indices[1])
            b = max(qubit_indices[0],qubit_indices[1])
            if(qubit_indices[0]!=0 and qubit_indices[1]!=0):
              list_circ = list(range(a,b+1))
              print(list_circ)
              list_len = len(list_circ)
              two_qubit_circuit = convert_two_qubit_gate(gate_name, qubit_indices[0]-1, qubit_indices[1]-1,list_len,params)
              new_circuit = new_circuit.compose(two_qubit_circuit, qubits=list_circ)
            elif(qubit_indices[0]==0 or qubit_indices[1]==0):
              list_circ = list(range(a,b+1))
              print(list_circ)
              list_len = len(list_circ)
              two_qubit_circuit = convert_two_qubit_gate(gate_name, qubit_indices[0], qubit_indices[1],list_len,params)
              new_circuit = new_circuit.compose(two_qubit_circuit, qubits=list_circ)
        elif len(qubit_indices) ==circuit.num_qubits:
              new_circuit.barrier()
        else:
            raise ValueError(f"Gate '{gate_name}' with {len(qubit_indices)} qubits is not supported.")

    return new_circuit
